# Detección de colisiones - DEMO

En este cuaderno se utilizará un modelo de machine-learning entrenado previamente para detectar si el JetBot se encuentra libre o bloqueado. Esto nos permitirá implementar la lógica que impida que el JetBot evite colisiones.

## Cargar el modelo preentrenado

El modelo a utilizar se debe encontrar en el mismo directorio de los ``notebooks``, con nombre ``best_model.pth``. Existen cuadernos adicionales con información sobre como entrenar este modelo desde cero. Es posible importar ficheros desde la propia interfaz de Jupyter.

> Asegurarse de que el fichero se ha importado antes de ejecutar las celdas siguientes.

La siguiente celda inicializa el modelo utilizando la librería PyTorch. PyTorch es un framework de machine learning que permite el entrenamiento y manipulación de modelos. En clases posteriores se profundizará en estos conceptos, la práctica actual está centrada en la utilización de un modelo previo que nos permita implementar nuevos comportamientos en el JetBot a partir de sus resultados.

Para simplificar, podemos considerar el modelo como una función que toma como entrada imágenes extraídas de la cámara y produce un resultado 'Bloqueado' o 'Libre', a partir de unos parámetros de configuración conocidos como ``pesos`` que son calculados en tiempo de entrenamiento.


In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

A continuación, cargar los pesos del fichero ``best_model.pth`` que acabamos de importar.

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

Actualmente los pesos del modelo se encuentran en la memoria de la CPU. La siguiente celda realiza una transferencia del modelo a la memoria de la GPU, mucho más rápida.

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### Función de preprocesado
El formato en el que fue entrenado el modelo no se corresponde exactamente con el formato de imágenes de la cámara, por lo que es necesario implementar una función de preprocesado que realice las siguientes transformaciones:

1. Transformación de BGR a RGB
2. Conversión de representación HWC a CHW 
3. Normalizar utilizando los parámetros que se utilizaron en tiempo de entrenamiento (la cámara proporciona valores en el rango [0, 255] y las imágenes de entrenamiento en el rango [0,1], por lo que realizará un escalado de 255).
4. Transferencia de datos entre CPU y GPU
5. Añadir dimensión adicional

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Ya se ha definido la función de preprocesamiento que convierte imágenes del formato de la cámara al formato de entrada de la red neuronal.

La siguiente celda muestra la cámara del JetBot y un slider que muestra la probabilidad de que el JetBot se encuentre bloqueado.

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

A continuación se crea una instancia del robot que nos permitirá interactuar con los motores.

In [ ]:
from jetbot import Robot

robot = Robot()

A continuación, se creará una función que será invocada cada vez que el valor de la cámara cambia. Esta función realizará los siguientes pasos:

1. Preprocesar la imagen de la cámara
2. Ejecutar red neuronal
3. Mientras la salida de la red neuronal indique que existe un bloqueo, se girará a la izquierda. En caso contrario se avanzará.

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # normalizar el vector de la red neuronal de manera que sume 1 (distribución de probabilidad)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # se invoca la primera vez para inicializar

Una vez definida la función, es necesario asociarla a la cámara del JetBot para su utilización. Podemos realizarlo con la función ``observe``

> OJO: La siguiente celda hará que el JetBot se mueva.

In [ ]:
camera.observe(update, names='value')  # asocia la función 'update' con la variable 'value' de la cámara

El Jetbot debería estar ejecutando acciones con cada frame de la cámara. Comprobar el correcto funcionamiento poniendo obstáculos en su camino. También debería detectar como bloqueo cuando se puede caer de la mesa.

Para detener este funcionamiento, es posible desconectar nuestra función de la siguiente manera.

In [ ]:
camera.unobserve(update, names='value')
robot.stop()

### Conclusión
El presente cuaderno muestra un ejemplo de como utilizar los resultados de un modelo machine learning  pre-entrenado para implementar nueva lógica en el comportamiento del robot.